In [1]:
import pandas as pd
import quant_utils.data_moudle as dm
from quant_utils.constant_varialbles import TODAY


In [2]:
file_path = 'E:/基金投顾自动化/基金投顾业务实时数据.xlsx'
df= pd.read_excel(file_path, sheet_name='基金投顾客户级明细报表')
file_path = 'E:/基金投顾自动化/小黑工评分表.xlsx'
person_df = pd.read_excel(file_path, sheet_name='分工')
portfolio_df = dm.get_portfolio_info()[["PORTFOLIO_NAME", "PORTFOLIO_TYPE", "DELISTED_DATE"]]
df = (
    df
    .merge(portfolio_df, left_on="组合名称", right_on="PORTFOLIO_NAME", how="left")
    .merge(person_df)
)
df["委托时间"] = pd.to_datetime(df["委托时间"])
df["培训日期"] = pd.to_datetime(df["培训日期"])

d:\envs\my311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


总量

In [3]:
df1 = df.copy()
df1 = df1.query("委托时间 >= 培训日期")
# df1 = df1.query("PORTFOLIO_TYPE != '短期周转'")
print("总量")
print(
    df1.groupby(by=["片区经理2"])["委托金额"].sum().reset_index().sort_values(by=["委托金额"], ascending=False)
)
print(
    df1.groupby(by=["片区经理"])["委托金额"].sum().reset_index().sort_values(by=["委托金额"], ascending=False)
)

总量
  片区经理2         委托金额
1   陈娇君  37565139.81
2   陈恺寅  23308434.72
0   陆天琦  13155431.00
  片区经理         委托金额
0  张晓军  40199103.63
3  马毓莹  14022326.54
1  王佶嘉  12185744.68
2  王贺歆   7621830.68


短债月增长

In [4]:
df1 = df.copy()
df1 = df1.query("委托时间 >= '2024-06-01' and 组合名称 == '知己优选-短债增强'")
df1 = df1.groupby(by=["分公司"])["委托金额"].sum().reset_index()
df1.sort_values(by="委托金额", ascending=False)

,分公司,委托金额
16,浙江分公司,11467971.82
0,三明分公司,10883735.21
15,泉州分公司,8352569.75
23,龙岩分公司,6017200.00
21,莆田分公司,5483200.85
19,福州分公司,5333014.55
11,广东分公司,2151183.00
14,江西分公司,1911000.00
4,南平分公司,1816000.00
18,漳州分公司,1374678.00


In [5]:
print(f'短债增强本月{df1["委托金额"].sum()/10000:.2f}/6000,进度完成{df1["委托金额"].sum()/10000/6000:.2%}')

短债增强本月6135.42/6000,进度完成102.26%


当日新增

In [6]:
df1 = df.copy()
df1 = df1.query(f"委托时间 >= '{TODAY}'")
print(f"{TODAY}新增")
print(
    df1.groupby(by=["片区经理2"])["委托金额"].sum().reset_index().sort_values(by=["委托金额"], ascending=False)
)
print(
    df1.groupby(by=["片区经理"])["委托金额"].sum().reset_index().sort_values(by=["委托金额"], ascending=False)
)

20240702新增
  片区经理2        委托金额
1   陈娇君  2885921.45
0   陆天琦   641000.00
2   陈恺寅   203000.00
  片区经理        委托金额
1  王佶嘉  2313721.45
3  马毓莹  1123100.00
0  张晓军   203100.00
2  王贺歆    90000.00


In [7]:
print(f"今日完成基金投顾总销量{df1['委托金额'].sum()/10000:.2f}万元")
temp = df1.groupby(by=["组合名称"])["委托金额"].sum().reset_index().sort_values(by=["委托金额"], ascending=False)

for idx, val in temp.iterrows():
    print(f'{val["组合名称"]}:{val["委托金额"]/10000:.2f}万元')

今日完成基金投顾总销量372.99万元
知己优选-月月享:150.00万元
知己优选-货币小福星:137.42万元
知己目标盈035期:52.60万元
知己优选-短债增强:32.37万元
知己优选-进取全明星:0.60万元


短债

In [8]:
df1 = df.copy()
df1 = df1.query("委托时间 >= '2024-06-01'")
df1 = df1.query("组合名称 == '知己优选-短债增强'")
df1["委托金额"].sum()/10000

6135.4244579999995

目标盈

In [9]:
df1 = df.copy()
df1 = df1.query("委托时间 >= '2024-06-01'")
df1 = df1.query("PORTFOLIO_TYPE == '目标盈'")
df1["委托金额"].sum()/10000

6098.27747

In [10]:
df2 = df.copy()
# df2 = df2[pd.notnull(df2["DELISTED_DATE"])]
df2 = df2.groupby(by=["PORTFOLIO_TYPE"])["委托金额"].sum().reset_index()
df2 = df2.sort_values(by=["委托金额"], ascending=False)
df2["委托金额"] = df2["委托金额"]/10000


In [11]:
df2["委托金额"].sum()

70427.307694

In [12]:
df3 = df.copy()
df3 = df3.query("PORTFOLIO_TYPE == '目标盈'")
df3.groupby("组合名称")["委托金额"].sum().reset_index().sort_values(by=["组合名称"], ascending=False)

,组合名称,委托金额
20,知己目标盈035期,705000.00
19,知己目标盈034期,12767367.45
18,知己目标盈033期,17548681.68
17,知己目标盈032期,12207141.70
16,知己目标盈031期,17752583.87
15,知己目标盈030期,24265466.84
14,知己目标盈029期,32839744.48
13,知己目标盈028期,21435029.12
12,知己目标盈027期,30865958.20
11,知己目标盈026期,14340804.75
